# General (Start here)


## Imports - BeatifulSoup, Selenium, ... 

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#from selenium.common.exceptions import NoSuchElementException

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 5.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 https:/

In [ ]:
import collections
from bs4 import BeautifulSoup
import requests
import string
import re
import json
import time

## Tags List

In [ ]:
all_tags = ["fraud", "hacker groups", "government", "corporation", "darknet", "incident response", "cyber intelligence", "antivirus", 
            "forensics", "pen testing", "firewall", "cyber defense", "hacking", "white hat", "black hat", "stenography", "cryptography",
            "cloud security", "botnet", "IoT", "DDOS", "network security","cyberwar", "usa", "russia", 
            "ukraine", "cyberterrorism", "data breach", "security breach", "vulnerability", "cve", "XSS", "patch", "mobile", "OS", "platform",
            "cyber attack", "malware", "virus", "phishing", "adware", "rootkit", "backdoor", "keylog", "trojan", 
            "ransomware", "spyware"]

# Gizmodo (Crawler, Search)

## Crawler

In [ ]:
from bs4 import BeautifulSoup
import requests
import string
import re

header = {
  "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36"
}

url1 = "https://gizmodo.com/china-mysterious-spaceplane-ejects-unknown-object-1849733129"

def scrapeGizmodo(url):
  html_text = requests.get(url, headers=header).text
  soup = BeautifulSoup(html_text, "lxml")
  dic = {}
  textContainer = soup.find("div", class_="xs32fe-0 iOFxrO js_post-content")
  if (not textContainer == None):
    pArray = textContainer.find_all("p")
    #print(pArray)
    raw_text = getText(pArray, ['<span>(.*?)">', '<\/a><\/span>', '<p.*?">', '<\/p>,', '<.*?>'])
    dic["raw text"] = raw_text
  return dic

def getText(text, regexList):
  newText = text
  for regex in regexList:
    newText = re.sub(regex, '', str(newText))
  return newText

In [ ]:
scrapeGizmodo(url1)["raw text"]

'[The saga of China’s spaceplane continues as the experimental vehicle just released a mystery object that’s now closely trailing behind in low Earth orbit.   On Monday, the United States Space Force’s 18th Space Defense Squadron tracked an object in a similar orbit to the spaceplane, SpaceNews first reported. The object appeared to be very close to the spaceplane. So close, in fact, that the Space Force unit had to make sure it was a separate object before it was entered into the database as such. The object may have been ejected earlier from the spaceplane, perhaps between October 24 to 30, but it got added to the database on October 31, according to a tweet by Robert Christy from Orbital Focus. Jonathan McDowell, an astronomer at the Harvard-Smithsonian Center for Astrophysics, suggested that the released object may be a service module, which houses support systems for its accompanying spacecraft. The object’s recent ejection may be “possibly indicating an upcoming deorbit burn,” Mc

## Search

In [ ]:
from bs4 import BeautifulSoup
import requests
import string
import re
import json
import time

driver = webdriver.Chrome('chromedriver',options=chrome_options)
search_gizmodo_dic = collections.defaultdict(list)

def end_of_page_check(soup, min_year, news_content):
  try:
    if(soup.find("a", {"class": "peggds-2 ixaYpK next-button"})):
      return False
    else:
      date_year = re.search('datetime="(\d*?)-', str(news_content))
      if date_year is not None:
        if(int(date_year.group(1)) > min_year):
          return False
      return True
  except:
    print("Error in eop check")
    return True

def search_gizmodo(keyword, min_year, max_page):#usage: search_gizmodo("hacking", 2019, 4)
  end_of_page = False;
  page_count = 1
  dic_list = []

  while(not end_of_page):
    #goto url
    print(page_count)
    keyword = keyword.replace(' ', '%20')
    if(page_count == 1):
      url = "https://gizmodo.com/tag/" + keyword
      #url = "https://gizmodo.com/search?blogId=4&q=" + china%20hacked"
    else:
      url = "https://gizmodo.com/tag/" + keyword + "?startIndex=" + str(page_count * 20)
    driver.get(url)
    driver.maximize_window()
    time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'lxml')
    news_content = soup.find_all("div", {"class": "cw4lnv-5 aoiLP"}) #get all news in page

    if(max_page <= page_count):
      end_of_page = True
    if(len(news_content) != 0):
      if(end_of_page_check(soup, min_year, news_content)):#is this the last page to iterate
        end_of_page = True

      for news in news_content:#get urls from news
        dic = {}
        news_url = re.search('"(https.*?)"', str(news))
        #print(news_url.group(1))
        if(news_url):
          if keyword not in search_gizmodo_dic[news_url.group(1)]:
            search_gizmodo_dic[news_url.group(1)].append(keyword)
    page_count += 1
      #print("page:" + str(page_count))

  #print("End, page count: " + str(page_count))

In [ ]:
search_gizmodo("hacking", 2017, 5)

# CNET (Crawler)

In [ ]:
from bs4 import BeautifulSoup
import requests
import string
import re

header = {
  "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36"
}

url1 = "https://www.cnet.com/tech/home-entertainment/apple-tv-4k-2022-review-in-progress-cheaper-price-new-chip-only-go-so-far/"

def scrapeCnet(url):
  html_text = requests.get(url, headers=header).text
  soup = BeautifulSoup(html_text, "lxml")
  dic = {}
  textContainer = soup.find("div", class_="col-6 article-main-body row")
  pArray = textContainer.find_all("p")
  raw_text = getText(pArray, ['<.*?>'])
  dic["raw text"] = raw_text
  #print(dic["raw text"])
  return dic

def getText(text, regexList):
  newText = text
  for regex in regexList:
    newText = re.sub(regex, '', str(newText))
  return newText


# CNN (Crawler)

In [ ]:
from bs4 import BeautifulSoup
import requests
import string
import re

header = {
  "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36"
}

url1 = "https://edition.cnn.com/2022/11/02/politics/biden-speech-democracy-dc/index.html"

def scrapeCnn(url):
  html_text = requests.get(url, headers=header).text
  soup = BeautifulSoup(html_text, "lxml")
  dic = {}
  textContainer = soup.find("div", class_="article__content")
  pArray = textContainer.find_all("p")
  raw_text = getText(pArray, ['<.*?>', '\\n'])
  dic["raw text"] = raw_text
  #print(dic["raw text"])
  return dic

def getText(text, regexList):
  newText = text
  for regex in regexList:
    newText = re.sub(regex, '', str(newText))
  return newText


# RealInfoSec (Crawler)

In [ ]:
from bs4 import BeautifulSoup
import requests
import string
import re

header = {
  "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36"
}

url1 = "https://www.realinfosec.net/cybersecurity-news/cosmiss-vulnerability-found-in-microsoft-azure-developer-tool/"

def scrapeRIS(url):
  html_text = requests.get(url, headers=header).text
  soup = BeautifulSoup(html_text, "lxml")
  dic = {}
  textContainer = soup.find("div", class_="entry-content clear")
  pArray = textContainer.find_all("p")
  raw_text = getText(pArray, ['<.*?>'])
  dic["raw text"] = raw_text
  #print(dic["raw text"])
  return dic

def getText(text, regexList):
  newText = text
  for regex in regexList:
    newText = re.sub(regex, '', str(newText))
  return newText


# TechCrunch (Crawler)

In [ ]:
from bs4 import BeautifulSoup
import requests
import string
import re

header = {
  "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36"
}

url1 = "https://techcrunch.com/2022/11/02/eric-schmidt-backs-former-google-execs-digital-family-office-platform-in-90-million-funding/"

def scrapeTechCrunch(url):
  html_text = requests.get(url, headers=header).text
  soup = BeautifulSoup(html_text, "lxml")
  dic = {}
  textContainer = soup.find("div", class_="article-content")
  pArray = textContainer.find_all("p")
  raw_text = getText(pArray, ['<.*?>'])
  dic["raw text"] = raw_text
  #print(dic["raw text"])
  return dic

def getText(text, regexList):
  newText = text
  for regex in regexList:
    newText = re.sub(regex, '', str(newText))
  return newText


# ITSecurityGuru (Search / Crawler) 

## Crawler

In [ ]:
from bs4 import BeautifulSoup
import requests
import string
import re

header = {
  "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36"
}

url1 = "https://www.itsecurityguru.org/2022/12/09/dragos-announces-partnership-with-cisco/"

def scrapeITGuru(url):
  html_text = requests.get(url, headers=header).text
  soup = BeautifulSoup(html_text, "lxml")
  dic = {}
  textContainer = soup.find("div", class_="content-inner ")
  raw_text = ""
  pArray = textContainer.find_all("p")
  for paragraph in pArray:
    raw_text += paragraph.get_text()
  #print(raw_text)
  dic["raw text"] = raw_text
  return dic

## Search

In [ ]:
search_dic = collections.defaultdict(list)
date_dic = collections.defaultdict(list)

def check_eop(html_text, max_year):#end of page
  date_regex = 'class="fa fa-clock-o"><\/i>(.*, (\d*))<\/a><\/div>'
  date = re.search(date_regex, html_text)
  if date is not None:
    date_year = date.group(2)
    if int(date_year) >= max_year:
      return False, date_year
    else:
      return True, 0
  return False, 0

def getSearchResults():
  
  queries1 = ['fraud', 'hacker groups', 'government', 'corporation',
       'darknet', 'cyber defense', 'hacking', 'security concepts',
       'security products', 'network security', 'cyberwar', 'geopolitical',
       'data breach', 'vulnerability', 'platform', 'cyber attack']

  queries=['fraud']
  for query in all_tags:
    print("Searching: " + query)

    driver = webdriver.Chrome('chromedriver',options=chrome_options)
    driver.maximize_window()
    current_page = 1
    end_of_page = False
    url= 'https://www.itsecurityguru.org/page/' + str(current_page) + '/?s=' + query #get first page
    driver.get(url)
    time.sleep(5)

    regex_last_page = '(<a class="page_number").*"(\d*)".*(\s<a class="page_nav next")'
    last_page = re.search(regex_last_page, str(driver.page_source))#get last page

    if last_page is not None:
      last_page = last_page.group(2)
      print("last_page: " + str(last_page))
      for page_no in range(1, int(last_page) + 1):#loop all pages
        if not end_of_page:
          print("current page: " + str(page_no))
          url= 'https://www.itsecurityguru.org/page/' + str(page_no) + '/?s=' + query
          driver.get(url)

          soup = BeautifulSoup(driver.page_source, 'lxml')
          news_list = soup.find_all("article", {"class": "jeg_post jeg_pl_md_2 format-standard"})

          for new in news_list:
            #print(len(news_list))
            date_result = check_eop(str(new), 2017)
            if not date_result[0]:#check date
              href_regex = '<a href=.*("https.*\/")><div class'
              web_url = re.search(href_regex, str(new))#get url
              if web_url is not None:
                web_url = web_url.group(1)
                if query not in search_dic[web_url]:#check unique
                  search_dic[web_url].append(query)
                  date_dic[web_url].append(date_result[1])
            # else:
            #   print("END OF PAGE")
            #   end_of_page = True

    driver.close()

def getText(text, regexList):
  newText = text
  for regex in regexList:
    newText = re.sub(regex, '', str(newText))
  return newText

In [ ]:
getSearchResults()

# BleepingComputer (Crawler / Search)

## Crawler (Not complete?)

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.tokenize import sent_tokenize, word_tokenize
# from nltk.stem import PorterStemmer
# porter=PorterStemmer()

from nltk.stem.snowball import SnowballStemmer
englishStemmer=SnowballStemmer("english")

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

from keras.preprocessing.text import Tokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import string
header = {
  "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36"
}

def removeStopWords(sentence):
  stop_words = set(stopwords.words('english'))
  word_tokens = word_tokenize(sentence)
  filtered_sentence = [w.lower() for w in word_tokens if not w.lower() in stop_words]

  # filtered_sentence to single string
  newSentence = ""

  for word in filtered_sentence:
    newSentence += word + " "

  newSentence = newSentence.translate(newSentence.maketrans("", "", string.punctuation + "“”’0123456789"))

  return newSentence


# tokenize the sentence
def tokenize(sentence):
  
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(sentence)
  sequences = tokenizer.texts_to_sequences(sentence)


  word_index = tokenizer.word_index
  bow = {}
  for key in word_index:
      bow[key] = sequences[0].count(word_index[key])

  return bow

def stemSentence(sentence):
  token_words=word_tokenize(sentence)
  stem_sentence=[]
  for word in token_words:
    w = (wordnet_lemmatizer.lemmatize(word, pos="n"))
    stem_sentence.append(englishStemmer.stem(w))
    stem_sentence.append(" ")
  return "".join(stem_sentence)

def mostFrequent(List):
  counter = 0
  num = List[0]
    
  for i in List:
    curr_frequency = List.count(i)
    if(curr_frequency> counter):
      counter = curr_frequency
      num = i

  return num

def mostFrequentList(List):
  counter = 0
  num = []
    
  for i in List:
    curr_frequency = List.count(i)
    if(curr_frequency> counter):
      counter = curr_frequency
      num = [i]
    elif(curr_frequency == counter and i not in num):
      num.append(i)

  return num

def scrapeAll(url):

  newsList = []
  html_text = requests.get(url, headers = header).text
  soup = BeautifulSoup(html_text, "lxml")
  allDivs = soup.find_all("a") # find all "a" tags
  # allDivs = soup.find_all("div") # find all "div" tags

  classListArr = []

  for div in allDivs: 
    # if(div.attrs.get("class") and len(div.text.strip()) > 0 and div.attrs.get("class")[0].find("image") == -1 and div.attrs.get("class")[0].find("img") == -1):
    if(div.attrs.get("class") and len(div.text.strip()) > 0):
      classListArr.append(div.attrs.get("class")[0])


  # print(classListArr)
  commonClass = mostFrequent(classListArr) # get the most common class name
  # print(commonClass)
  allNews = soup.find_all("a", class_ = commonClass) # get all the "a" tags with most common class name
  # allNews = soup.find_all("a", class_ = commonClass) # get all the "div" tags with most common class name

  # for new in allNews:
  #   print(new)

  # print(allTitles)


  for news in allNews:
    dic = {}
    newsText = news.text.strip()
    newsLink = news.get("href")
    if(newsLink.find("http") == -1): # if link is broken
      newUrl = url
      if(len(url.split("/")) > 3):
        # newUrl = url[:url.rfind("/")]
        newUrl = url.split("/")[0] + "//" + url.split("/")[2]
      newsLink = newUrl + newsLink

    # dic["text"] = newsText
    dic["header"] = newsText
    dic["link"] = newsLink
    newsList.append(dic)
    # print(f'text: {newsText} \nlink: {newsLink}')
    # print(f'text: {newsText}')
    # print(f'link: {newsLink}\n')

  return newsList

def take_second(elem):
    return elem[1]

def sortWordOccurences(sentence):
  x = removeStopWords(sentence)
  x = x.translate(x.maketrans("", "", string.punctuation + "“”’0123456789"))
  x= tokenize([x])
  x = sorted(x.items(), reverse = True, key = take_second)

  return x

In [ ]:


def take_second(elem):
    return elem[1]

def sortWordOccurences(sentence):
  x = removeStopWords(sentence)
  x = x.translate(x.maketrans("", "", string.punctuation + "“”’0123456789"))
  x= tokenize([x])
  x = sorted(x.items(), reverse = True, key = take_second)

  return x

In [ ]:
categoryMap = {"security": 1, "non-security": 0}

# df = pd.DataFrame(columns = ["title", "first two paragraph", "url", "date", "stemmed text", "category"])
mainDf = pd.DataFrame()

In [ ]:
# BLEEPINGCOMPUTER.COM

def tokenizeBleepingComputer(url):
  data = requests.get(url, headers = header).text
  new_soup = BeautifulSoup(data, "lxml")
  body = new_soup.find_all("div", class_ = "articleBody")

  newsText = ""


  for txt in body:
    newsText += txt.text.strip();

  newsText = newsText.translate(newsText.maketrans("", "", string.punctuation + "“”0123456789"))

  sentence = removeStopWords(newsText)
  sentence = stemSentence(sentence)
  return sentence
  # return sentence.split(" ")
  # return tokenize([sentence])

def scrapeBleepingComputer(page):
  html_text = requests.get('https://bleepingcomputer.com/' + page, headers = header).text
  soup = BeautifulSoup(html_text, "lxml")
  # print(soup)
  allNews = soup.find_all("div", class_ = "bc_latest_news_text")
  
  df = pd.DataFrame()

  for news in allNews:
    dic = {}
    date = news.ul.contents[2].text
    category = news.div.a.text
    title = news.h4.a
    firstParagraph = news.p.text
    title_text = title.text
    title_link = title["href"]
    dic["title"] = title_text
    dic["first paragraph"] = firstParagraph
    dic["url"] = title_link
    dic["date"] = date
    if(category == "Security"):
      dic["category"] = "security"
    else:
      dic["category"] = "non-security"
    dic["stemmed text"] = tokenizeBleepingComputer(title_link)
    df = df.append(dic, ignore_index = True)
  return df

In [ ]:
for i in range(100):
  if(i == 0):
    df = scrapeBleepingComputer("")
  else:
    df = scrapeBleepingComputer("page/" + str(i + 1) + "/")
  mainDf = mainDf.append(df, ignore_index= True)

LookupError: ignored

## Search

In [ ]:
driver = webdriver.Chrome('chromedriver',options=chrome_options)
import collections
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
from bs4 import BeautifulSoup
import requests
import string
import re
import json
import collections

In [ ]:
search_dic = collections.defaultdict(list)
html_list = []
len(all_tags)

In [ ]:
def getSearchResults(tag_list):
  queries1 = ['fraud', 'hacker groups', 'government', 'corporation',
       'darknet', 'cyber defense', 'hacking', 'security concepts',
       'security products', 'network security', 'cyberwar', 'geopolitical',
       'data breach', 'vulnerability', 'platform', 'cyber attack']

  queries=['fraud', 'hacking']
  
  driver = webdriver.Chrome('chromedriver',options=chrome_options)
  driver.maximize_window()

  for tag in tag_list:
    print(len(search_dic))
    print(search_dic) 
    print("Searching: " + tag)
    
    url = 'https://www.bleepingcomputer.com/search/?cx=partner-pub-0920899300397823%3A3529943228&cof=FORID%3A10&ie=UTF-8&q='+ tag
    driver.get(url)
    time.sleep(3)

    current_page = 1
    for i in range(1, 11):
      print("Page:", current_page)
      soup = BeautifulSoup(driver.page_source, 'lxml')
      all_news = soup.find_all('div', {"class": "gsc-webResult gsc-result"})
      for new in all_news:
        span = new.find('div', {"class": "gs-bidi-start-align gs-visibleUrl gs-visibleUrl-breadcrumb"})#for span News
        if check_span(span.text):
          date = new.find('div', {"class": "gs-bidi-start-align gs-snippet"})
          if check_date(date.text, 2017):
            href = new.find('a', {"class": "gs-title"})
            if href['href'] is not None:
              print("url", href['href'])
              web_url = href['href']
              if tag not in search_dic[web_url]:
                search_dic[web_url].append(tag)
                
      time.sleep(1)
      current_page = current_page + 1
      if current_page <= 10:
        #button = driver.find_element(By.CSS_SELECTOR, 'div.gsc-cursor-page:nth-child(' + str(current_page) + ')')
        button = WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.gsc-cursor-page:nth-child(' + str(current_page) + ')')))
        if button is not None:
          driver.execute_script("arguments[0].click();", button)
          time.sleep(3)


def check_date(html_text, min_year):
  if(html_text):
    recent =  re.search('days ago', html_text[0:20])
    if recent is not None:
      return True
    else:
      date = re.search(r"\d, (\b\d{4}\b)", html_text[0:20])
      if date is not None:
        date_year = date.group(1)
        if int(date_year) >= min_year:
          return True
  return False

def check_span(html_text):
  if(html_text):
    element = re.search('.* › (.*?) › .*', html_text)
    if element is not None:
      element_value = element.group(1)
      if element_value == "News":
        return True
  return False

def parsehtml(html_text):
  soup = BeautifulSoup(html_text, 'lxml')
  html_list.append(html_text)
  all_news = soup.find_all('div', {"class": "gsc-webResult gsc-result"})#get all news in the page
  for new in all_news:
    span = new.find('div', {"class": "gs-bidi-start-align gs-visibleUrl gs-visibleUrl-breadcrumb"})#check if post type is news
    if span is not None: 
      if check_span(span.text):
        date = new.find('div', {"class": "gs-bidi-start-align gs-snippet"})
        if date is not None:
          if check_date(date.text, 2017):
            href = new.find('a', {"class": "gs-title"})
            if href['href'] is not None:
              print("url", href['href'])
              web_url = href['href']
              if tag not in search_dic[web_url]:
                search_dic[web_url].append(tag)

In [ ]:
getSearchResults(all_tags)

# SCMagazine (Search / Crawler)

## Search

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
search_dic = collections.defaultdict(list)
date_dic = collections.defaultdict(list)
all_dic = collections.defaultdict(list)

def check_date(min_year, html_text):
  date_regex = 'itemprop="datePublished">.*, (.*?)<\/time>'
  #date_regex = 'datetime=\"(.*?)-.*\" itemprop'
  date_year = re.search(date_regex, html_text)
  #print(html_text)
  if date_year is not None:
    date_year = int(date_year.group(1))
    if date_year >= min_year:
      return True, date_year
  else:
    print("date zort")
  return False, 0

def check_article_type(html_text):
  articleType_regex = 'searchTeaserCard_label__Szqey">(.*?)<\/div>'
  article_type = re.search(articleType_regex, html_text)
  if article_type is not None:
    article_type = article_type.group(1)
    if article_type in ["brandview", "news", "brief", "analysis", "resource", "feature"]:
      print(article_type)
      return True, article_type
  return False, ""
#\/(.*?)\/.*

def check_article_type2(web_url):
  articleType_regex = '\/(.*?)\/.*'
  article_type = re.search(articleType_regex, web_url)
  if article_type is not None:
    article_type = article_type.group(1)
    if article_type in ["brandview", "news", "brief", "analysis", "resource", "feature"]:
      return True, article_type
  return False, ""

def getSearchResults():
  
  queries1 = ['fraud', 'hacker groups', 'government', 'corporation',
       'darknet', 'cyber defense', 'hacking', 'security concepts',
       'security products', 'network security', 'cyberwar', 'geopolitical',
       'data breach', 'vulnerability', 'platform', 'cyber attack']

  queries=['darknet']
  for query in queries:
    print("Searching: " + query)

    driver = webdriver.Chrome('chromedriver',options=chrome_options)
    driver.maximize_window()

    url = 'https://www.scmagazine.com/search?q=' + query
    driver.get(url)
    time.sleep(5)  # Allow 2 seconds for the web page to open
    scroll_pause_time = 3 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
    i = 1

    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(scroll_pause_time)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height:
            break

    soup = BeautifulSoup(driver.page_source, 'lxml')
    news_list = soup.find_all("div", {"class": "teaser-list"})
    for news_group in news_list:
      for new in news_group:
        href_regex = 'class="font-sans".* href="(.*?)">'
        web_url = re.search(href_regex, str(new))#get url
        if web_url is not None:
          web_url = web_url.group(1)

          if query not in all_dic['https://www.scmagazine.com' + web_url]:
            all_dic['https://www.scmagazine.com' + web_url].append(query)

          article_type_result = check_article_type2(web_url)
          if article_type_result[0]:
            date_result = check_date(2017, str(new))
            if date_result[0]:
              if query not in search_dic['https://www.scmagazine.com' + web_url]:#check unique tag
                search_dic['https://www.scmagazine.com' + web_url].append(query)
                date_dic['https://www.scmagazine.com' + web_url].append(date_result[1])
                print('https://www.scmagazine.com' + web_url)
          
    driver.close()

In [ ]:
getSearchResults()

In [ ]:
print(len(search_dic))
print(len(all_dic))
print((search_dic))
print((all_dic))